In [1]:
!nvidia-smi

Fri Oct 13 15:14:37 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -qqq git+https://github.com/bofenghuang/vigogne

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.

In [3]:
# Uncomment if necessary
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
from typing import Dict, List, Optional
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, TextStreamer

In [4]:
model_name_or_path = "bofenghuang/vigogne-2-7b-chat"
revision = "v2.0"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, revision=revision, padding_side="right", use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    revision=revision,
    torch_dtype=torch.float16,
    device_map="auto",
    # load_in_8bit=True,
    # low_cpu_mem_usage=True,
)

streamer = TextStreamer(tokenizer, timeout=10.0, skip_prompt=True, skip_special_tokens=True)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [7]:
def chat(
    query: str,
    history: Optional[List[Dict]] = None,
    temperature: float = 0.7,
    top_p: float = 1.0,
    top_k: float = 0,
    repetition_penalty: float = 1.1,
    max_new_tokens: int = 1024,
    **kwargs,
):
    if history is None:
        history = []

    history.append({"role": "user", "content": query})

    input_ids = tokenizer.apply_chat_template(history, add_generation_prompt=True, return_tensors="pt").to(model.device)
    input_length = input_ids.shape[1]

    generated_outputs = model.generate(
        input_ids=input_ids,
        generation_config=GenerationConfig(
            temperature=temperature,
            do_sample=temperature > 0.0,
            top_p=top_p,
            top_k=top_k,
            repetition_penalty=repetition_penalty,
            max_new_tokens=max_new_tokens,
            pad_token_id=tokenizer.eos_token_id,
            **kwargs,
        ),
        streamer=streamer,
        return_dict_in_generate=True,
    )

    generated_tokens = generated_outputs.sequences[0, input_length:]
    generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)

    history.append({"role": "assistant", "content": generated_text})

    return generated_text, history

In [10]:
# 1st round
response, history = chat("Un escargot parcourt 100 mètres en 5 heures. Quelle est sa vitesse ?", history=None)

Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


Pour calculer la vitesse de l'escargot, nous devons d'abord convertir la distance (mètres) et le temps (heures) en unités communes. Nous savons déjà qu'un mètre équivaut à 3,28 pieds et qu'une heure vaut environ 3600 secondes.

La formule pour trouver la vitesse est : Vitesse = Distance / Temps

Dans ce cas, la distance est de 100 mètres et le temps est de 5 heures. La vitesse de l'escargot sera donc :

Vitesse = 100 m / 5 h
            ≈  20 km/h

Donc, la vitesse de l'escargot est d'environ 20 kilomètres par heure.


In [11]:
# 2nd round
response, history = chat("Quand il peut dépasser le lapin ?", history=history)

Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


Il n'est pas possible de dire avec certitude si un escargot peut "dépasser" un lapin ou non, car cela dépendrait de la situation spécifique dans laquelle ils se trouvent. Cependant, il est important de noter que les deux animaux sont des mammifères terrestres et auraient tous deux besoin d'espaces suffisants pour se déplacer rapidement.

En général, les lapins ont tendance à être plus rapides que les escargots sur courte distance, tandis que les escargots peuvent être plus adaptés aux longues distances à cause de leur capacité à marcher sur une surface plane et à avancer de manière continue sans interruption. Toutefois, les vitesse des deux animaux varieraient en fonction de nombreux facteurs, tels que la forme et l'état du sol, les conditions météorologiques et la fatigue.

Il est également utile de rappeler qu'il s'agit de questions de vitesse relative plutôt que de compétition absolue. Par exemple, un lapin pourrait considérer comme étant "plus rapide" qu'un escargot lorsqu'ils se d

In [12]:
# 3rd round
response, history = chat("Écris une histoire imaginative qui met en scène une compétition de course entre un escargot et un lapin.", history=history)

Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


Dans un petit village caché au cœur d'une forêt dense, vivait un lapin courageux nommé Passe-Pied. Le jour où il avait 6 ans, Passe-Pied avait rencontré un étrange personnage en forme d'escargot géant. Ce dernier lui avait proposé d'en apprendre davantage sur les mystères de la forêt. Passe-Pied accepta et bientôt, une amitié solide s'établit entre eux.

Un jour, alors que Passe-Pied traversait la forêt pour retrouver ses amis, il aperçut un grand concours de course. Le 

OutOfMemoryError: ignored